In [ ]:
pip install git+https://github.com/tblume1992/statsforecast.git@feature/mfles

  Cloning https://github.com/tblume1992/statsforecast.git (to revision feature/mfles) to /tmp/pip-req-build-rst06yu0
  Running command git clone --filter=blob:none --quiet https://github.com/tblume1992/statsforecast.git /tmp/pip-req-build-rst06yu0
  Running command git checkout -b feature/mfles --track origin/feature/mfles
  Switched to a new branch 'feature/mfles'
  Branch 'feature/mfles' set up to track remote branch 'feature/mfles' from 'origin'.
  Resolved https://github.com/tblume1992/statsforecast.git to commit a1568e0e996a9c3ff5ed3b3986c9154703bbb202
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 4.4 MB/s eta 0:0

In [ ]:
pip install mfles

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from statsforecast.models import MSTL, AutoETS, AutoTBATS, AutoMFLES
from MFLES.Forecaster import MFLES as old_MFLES
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:41: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def lasso_nb(X, y, alpha, tol=0.001, maxiter=10000):
/usr/local/lib/python3.10/dist-packages/MFLES/Model.py:164: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Hourly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Hourly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
mfles_smape = []
naive_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'New MFLES:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))
    mfles_model = AutoMFLES(
         season_length = [24, 168],
         n_windows = 3,
         test_size = 24,
         step_size = 24,
         metric = 'smape')
    mfles_model = mfles_model.fit(y=y.values)
    fitted = mfles_model.predict_in_sample()['fitted']
    predicted = mfles_model.predict(h=len(y_test), X=None)['mean']
    mfles_smape.append(smape(y_test, predicted.clip(min=0)))
print('/b')
print(f'Stats Forecast MFLES {np.nanmean(mfles_smape)}')
print(f'Naive {np.mean(naive_smape)}')

  0%|          | 0/414 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
MFLES:nan,New MFLES:nan, AutoTBATS:nan, Naive:nan:   0%|          | 1/414 [00:01<10:14,  1.49s/it]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
MFLES:nan,New MFLES:3.4287647275140545, AutoTBATS:nan, Naive:nan:   0%|          | 2/414 [00:03<10:21,  1.51s/it]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty 

/b
Old MFLES nan
MSTL nan
Stats Forecast MFLES 11.808786850332853
Naive nan


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Monthly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Monthly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
naive_smape = []
mfles_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'New MFLES:{np.mean(ets_smape)}, Naive:{np.mean(naive_smape)}')
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))
    mfles_model = AutoMFLES(
         season_length = [12],
         n_windows = 3,
         test_size = 12,
         step_size = 12,
         metric = 'smape')
    mfles_model = mfles_model.fit(y=y.values)
    fitted = mfles_model.predict_in_sample()['fitted']
    predicted = mfles_model.predict(h=len(y_test), X=None)['mean']
    mfles_smape.append(smape(y_test, predicted.clip(min=0)))
print('/b')
print(f'Stats Forecast MFLES {np.nanmean(mfles_smape)}')
print(f'Naive {np.mean(naive_smape)}')

Streaming output truncated to the last 5000 lines.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
MFLES:nan,New MFLES:12.427418049065585, AutoTBATS:nan, Naive:nan:  96%|█████████▋| 46229/48000 [5:16:58<10:28,  2.82it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
MFLES:nan,New MFLES:12.427246641962483, AutoTBATS:nan, Naive:nan:  96%|█████████▋| 46230/48000 [5:16:59<11:26,  2.58it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/loc

/b
Old MFLES nan
MSTL nan
Stats Forecast MFLES 12.83757521982694
Naive nan


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Weekly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Weekly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
naive_smape = []
mfles_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'New MFLES:{np.mean(mfles_smape)}, Naive:{np.mean(naive_smape)}')
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    mfles_model = AutoMFLES(
         season_length = [52],
         n_windows = 1,
         test_size = 26,
         metric = 'smape')
    mfles_model = mfles_model.fit(y=y.values)
    fitted = mfles_model.predict_in_sample()['fitted']
    predicted = mfles_model.predict(h=len(y_test), X=None)['mean']
    mfles_smape.append(smape(y_test, predicted.clip(min=0)))
print('/b')
print(f'Stats Forecast MFLES {np.mean(mfles_smape)}')
print(f'Naive {np.mean(naive_smape)}')

MFLES:8.351399120061986,New MFLES:8.262770904163977, Naive:9.13930537684544: 100%|██████████| 359/359 [09:43<00:00,  1.63s/it]

/b
Old MFLES 8.410276705227382
Stats Forecast MFLES 8.32189536458257
Naive 9.161286913982


In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Quarterly-train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Quarterly-test.csv')

train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [ ]:
naive_smape = []
mfles_smape = []
j = tqdm(range(len(train_df)))

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

for row in j:
    y = train_df.iloc[row, :].dropna()
    snaive = y.shift(1).dropna()
    snaive = snaive.values
    y_test = test_df.iloc[row, :].dropna().values
    j.set_description(f'New MFLES:{np.mean(mfles_smape)}, Naive:{np.mean(naive_smape)}')
    naive_smape.append(smape(y_test, np.tile(y.iloc[-1], len(y_test))))

    try:
      mfles_model = AutoMFLES(
          season_length = [4],
          n_windows = 3,
          test_size = 4,
          step_size=4,
          metric = 'smape')
      mfles_model = mfles_model.fit(y=y.values)
    except:
        mfles_model = AutoMFLES(
          season_length = [4],
          n_windows = 1,
          test_size = 4,
          step_size=4,
          metric = 'smape')
        mfles_model = mfles_model.fit(y=y.values)
    fitted = mfles_model.predict_in_sample()['fitted']
    predicted = mfles_model.predict(h=len(y_test), X=None)['mean']
    mfles_smape.append(smape(y_test, predicted.clip(min=0)))
print('/b')
print(f'Stats Forecast MFLES {np.mean(mfles_smape)}')
print(f'Naive {np.mean(naive_smape)}')